In [60]:
from blockchain import Blockchain, Block

blockchain = Blockchain()

genesis = blockchain.chain[0]
b = blockchain.generate_new_block()
print(genesis.__dict__, "\n\n")
print(b.__dict__)

{'index': 0, 'hash': '0000607e58623c07e2634999e38638e13b76af5b450cd7a295bc60179e6db208', 'previous_hash': '', 'timestamp': 'inthebeninging', 'proof': 33} 


{'index': 1, 'hash': '0000cd8127636708fa10c167d0b62d0dd428cec66b32e2e2c2c74efd466d154f', 'previous_hash': '0000607e58623c07e2634999e38638e13b76af5b450cd7a295bc60179e6db208', 'timestamp': 1639514663.2652752, 'proof': 36785}


# trying out signatures

In [61]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization

# waarom heb ik het gevoel dat het veel makkeljker dan dit kan

def makeKeyPair():
    private_key = rsa.generate_private_key(
                    public_exponent=65537,
                    key_size=512,
                  )

    public_key = private_key.public_key()

    return private_key, public_key

def signSomething(private_key, something):
    signature = private_key.sign(
        something,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    
    return signature

def verifySomething(public_key, signature, something):
    try:
        public_key.verify(
            signature,
            something,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
    except:
        print("Invalid Signature")


In [62]:
# making the key pairs
privkey, pubkey = makeKeyPair()
privkey2, pubkey2 = makeKeyPair()

# signing a message with the private key
message = b"A message I want to sign"
signature = signSomething(privkey2, message)

# verifying it
verifySomething(pubkey, signature, message)

Invalid Signature


# trying out transactions

In [89]:
from transaction import Transaction, TxIn, TxOut

COINBASE_AMOUNT = 20

bob_addr = "bob123"
alice_addr = "alice321"
bob_privkey, bob_pubkey = makeKeyPair()

# Coinbase transaction to Bob
coinbase_in = TxIn("", 0, "")
coinbase_out = TxOut(bob_addr, COINBASE_AMOUNT)
coinbase_transaction = Transaction([coinbase_in], [coinbase_out])

# Bob's gift to Alice
bob_in = TxIn(coinbase_transaction.id, 0, "")
bob_out1 = TxOut(alice_addr, 10)
bob_out2 = TxOut(bob_addr, 10)
bobs_gift_transaction = Transaction([bob_in], [bob_out1, bob_out2])

unsignedTxIns = bobs_gift_transaction.txIns
for uTxIn in unsignedTxIns:
    signature = signSomething(bob_privkey, bobs_gift_transaction.id.encode())
    uTxIn.signature = signature

# Verifying that thing is signed


In [165]:
# TODO
# hoe sla ik keys op
# wil de keys in string format hebben


pem = bob_pubkey.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
        )

pub_key_bytes = pem.splitlines()[1:-1]
pub_key_str = [x.decode() for x in pub_key_bytes] 
pub_key = "".join(pub_key_str)
print(pub_key)

pem2 = bob_pubkey.public_bytes(
        encoding=serialization.Encoding.DER,
        format=serialization.PublicFormat.PKCS1
        )

pem2.hex()

MFwwDQYJKoZIhvcNAQEBBQADSwAwSAJBANJITAWvGK94NuTl33bydUsEKxQlMATJEzduhuo+d9wR+PNiiz2SQeIorslQYCCE66l7JL8vmX00eugEn8bfQZcCAwEAAQ==


'3048024100d2484c05af18af7836e4e5df76f2754b042b14253004c913376e86ea3e77dc11f8f3628b3d9241e228aec950602084eba97b24bf2f997d347ae8049fc6df41970203010001'

In [162]:
from Crypto.PublicKey import RSA
key = RSA.generate(1024)

thebinary = key.exportKey('DER')

thebinary.hex()

'3082025c02010002818100ad109696e40658d8054f625f67b24a3b963b760b1ca5a2b5d9b9d041b9fa538fcc860ef506de008b5209b7e28397891f3e5bb72daa2afe00c3713f5d557e8fffca4e2824de21247e42cd863ac97f51864d0ff66fbacf1bd4cf9003bc703a5189b9fd6f81ae50a040cfe6d531192c86fe8443a377b3a6a5e1f8af1bf183faf853020301000102818032f335424af34b689d7797807ae8d44de296b2d3e6a524d7e0b781863818684d676af8304ecb40e1c34f3521ac980a3876854dd255ccbe66bf2332026a4f0e5944d724d1617cdb4a64c1b693117ac4e35731b31577e48efe9afaaed4c89e73fb174b7b3f7e2b908f4ea06525c8fb325e21d4b00d3613ca278cbd9e9f729729f1024100b89de4c1143f749789f2a5cf52b12b4cc32ceef2af27fca614c6b152e47aba7dc6d5e93b5657f997d318a7a85dea783df839540e9f3fee3027c2fe497ddb0edd024100effb3bed0f23ac7620446c12831ed7ba446d263aa33af908a65605b39e040c000b5ff6021282b64757d493832eff90af593cf1a1858a680a404ede16303af8ef024071a31eef8a1b151542204dd7b3bac24059d30ab1ef06da0910b8cce58d25ce2d257afe393c08a2e5c55663bccb324c1938f570bc1f6226a715927d1b97908331024100d5389ebcf9079432fc666eb36006e65bdc723256453